# Static as PNG

**Intent**

The image of an old black-and-white TV in horror films is ubiquitous.
This project aims to create digital snapshots of that experience both in black-and-white as well as in color. 

# Color Space

The color space computers generally use is [RGB](https://en.wikipedia.org/wiki/RGB).
RGB is a digital color space.
It has three discrete channels all of which are represented as integers ([0, 255]).

The color space old TVs used in the US was [NTSC](https://en.wikipedia.org/wiki/NTSC) (A.K.A. [YIQ](https://en.wikipedia.org/wiki/YIQ)).
YIQ is _not_ a digital color space.
It was _meant_ for analog.
This means there are going to be issues that crop up.
Some of those differences are presented below.

## Questions

1. Is the built-in color conversion reversible or near reversible?
2. Is the custom color conversion reversible or near reversible?
3. What part of the YIQ color space is not addressable as RGB

## Q1 - Pseudocode

1. Start with the digital RGB space (256 x 256 x 256)
2. Convert to analog YIQ space
3. Calculate the YIQ range
4. Reverse the YIQ range to RGB to check for (near)reversibility

In [1]:
import colorsys as cs

y_rng = [float('inf'), float('-inf')]
i_rng = [float('inf'), float('-inf')]
q_rng = [float('inf'), float('-inf')]
tot = 256 * 256 * 256
r1 = tot
r2 = tot
lim = 1/256

for r in range(256):
    for g in range(256):
        for b in range(256):
            (r_in, g_in, b_in) = (r/255, g/255, b/255)
            (y, i, q) = cs.rgb_to_yiq(r_in, g_in, b_in)
            (r_out, g_out, b_out) = cs.yiq_to_rgb(y, i, q)
            y_rng = (min(y_rng[0], y), max(y_rng[1], y))
            i_rng = (min(i_rng[0], i), max(i_rng[1], i))
            q_rng = (min(q_rng[0], q), max(q_rng[1], q))
            if not all([r_in == r_out, g_in == g_out, b_in == b_out]):
                r1 -= 1
            if not all([abs(r_in - r_out) <= lim, abs(g_in - g_out) <= lim, (b_in - b_out) <= lim]):
                r2 -= 1

print('--- using built-in color conversion ---')
print(f'Y: [{y_rng[0]}, {y_rng[1]}]')
print(f'I: [{i_rng[0]}, {i_rng[1]}]')
print(f'Q: [{q_rng[0]}, {q_rng[1]}]')
print(f'{100 * r1 / tot} % of the space is perfectly reversible ({r1}/{tot})')
print(f'{100 * r2 / tot} % of the space is near perfectly (diff <= {lim}) reversible ({r2}/{tot})')

del y_rng, i_rng, q_rng, tot, r1, r2, lim, r_in, g_in, b_in, r, g, b, y, i, q, r_out, g_out, b_out

## Q1 - Results

Using the built-in color space conversion:

* The RGB space must be [0, 1] for `colorsys.yiq_to_rgb()` to even make sense
* The RGB space [0, 1] x [0, 1] x [0, 1] converts to the YIQ space [0.0, 1.0] x [-0.599, 0.599] x [-0.5251, 0.5251]
* 12.99 % of the space is perfectly reversible
* 100 % of the space is near perfectly (difference <= 1/256) reversible

## Custom Color Space Functions

There is a built-in color package (`colorsys`) in Python.
There is a built-in image package (`PIL`) in Python.
Pixel values in `PIL` work in the range [0, 255]. 
`colorsys` does not allow for rgb => yiq => rgb in `PIL`'s range.

The math used in `rgb_to_yiq` and `yiq_to_rgb` are based on the formulas found [here](https://en.wikipedia.org/wiki/YIQ).

**NOTE**: The [clamp](https://en.wikipedia.org/wiki/Clamping_(graphics)) in `yiq_to_rgb` this is defined in the [standard](https://en.wikipedia.org/wiki/YUV).

In [2]:
import typing as t

def rgb_to_yiq(rgb: t.Tuple[int, int, int]) -> t.Tuple[float, float, float]:
    y = 0.2990 * rgb[0] + 0.5870 * rgb[1] + 0.1140 * rgb[2]
    i = 0.5959 * rgb[0] - 0.2746 * rgb[1] - 0.3213 * rgb[2]
    q = 0.2115 * rgb[0] - 0.5227 * rgb[1] + 0.3112 * rgb[2]
    return (y, i, q)

def yiq_to_rgb(yiq: t.Tuple[float, float, float]) -> t.Tuple[int, int, int]:
    r = round(max(0, min(255, 1 * yiq[0] + 0.956 * yiq[1] + 0.619 * yiq[2])))
    g = round(max(0, min(255, 1 * yiq[0] - 0.272 * yiq[1] - 0.647 * yiq[2])))
    b = round(max(0, min(255, 1 * yiq[0] - 1.106 * yiq[1] + 1.703 * yiq[2])))
    return (r, g, b)

## Q2 - Pseudocode

1. Start with the digital RGB space (256 x 256 x 256)
2. Convert to analog YIQ space
3. Calculate the YIQ range
4. Reverse the YIQ range to RGB to check for (near)reversibility

In [3]:
y_rng = [float('inf'), float('-inf')]
i_rng = [float('inf'), float('-inf')]
q_rng = [float('inf'), float('-inf')]
tot = 256 * 256 * 256
r1 = tot
r2 = tot
lim = 1

for r in range(256):
    for g in range(256):
        for b in range(256):
            (r_in, g_in, b_in) = (r, g, b)
            (y, i, q) = rgb_to_yiq((r_in, g_in, b_in))
            (r_out, g_out, b_out) = yiq_to_rgb((y, i, q))
            y_rng = (min(y_rng[0], y), max(y_rng[1], y))
            i_rng = (min(i_rng[0], i), max(i_rng[1], i))
            q_rng = (min(q_rng[0], q), max(q_rng[1], q))
            if not all([r_in == r_out, g_in == g_out, b_in == b_out]):
                r1 -= 1
            if not all([abs(r_in - r_out) <= lim, abs(g_in - g_out) <= lim, (b_in - b_out) <= lim]):
                r2 -= 1

print('--- using custom colors conversion ---')
print(f'Y: [{y_rng[0]}, {y_rng[1]}]')
print(f'I: [{i_rng[0]}, {i_rng[1]}]')
print(f'Q: [{q_rng[0]}, {q_rng[1]}]')
print(f'{100 * r1 / tot} % of the space is perfectly reversible ({r1}/{tot})')
print(f'{100 * r2 / tot} % of the space is near perfectly (diff <= {lim}) reversible ({r2}/{tot})')

del y_rng, i_rng, q_rng, tot, r1, r2, lim, r_in, g_in, b_in, r, g, b, y, i, q, r_out, g_out, b_out

## Q2 - Results

Using the custom color space conversion

* The RGB space is [0, 255]
* The RGB space [0, 255] x [0, 255] x [0, 255] converts to the YIQ space [0, 255] x [-151.9545, 151.9545] x [-133.2885, 133.2885]
* 100 % of the space is perfectly reversible

## Q3 - Pseudocode

1. Start with the analog YIQ space [0, 255] x [-151.9545, 151.9545] x [-133.2885, 133.2885]
2. Sample 4 x 256 x 256 x 256 times from the space.
   **Note**: This is a probalistic test.
3. Convert from  YIQ to RGB
4. Check to see if the RGB is outside the bounds of [0, 255]

In [4]:
from random import randint, uniform, seed

seed(0)

y_rng = [0, 255]
i_rng = [-151.9545, 151.9545]
q_rng = [-133.2885, 133.2885]
tot = 4 * 256 * 256 * 256
good = tot

for _ in range(tot):
    y = randint(y_rng[0], y_rng[1])
    i = uniform(i_rng[0], i_rng[1])
    q = uniform(q_rng[0], q_rng[1])
    (r, g, b) = yiq_to_rgb((y, i, q))
    if not all([0 <= r, r < 256, 0 <= g, g < 256, 0 <= b, b < 256]):
        good -= 1

print(f'{100 * good / tot} % of the YIQ space is addressable as RGB ({good}/{tot})')

del y_rng, i_rng, q_rng, tot, good, y, i, q, r, g, b

## Q3 - Results

* Thanks to clamping 100 % of the YIQ space is addressable as RGB

# Generate Static - YIQ/RGB (color + BW)

## Pseudo Code - YIQ

1. Generate random numbers
2. Interpret those values as NTSC values.
   NTSC uses YIQ as the color space
3. Convert YIQ to RGB
4. Save the images as a PNG
5. Repeat

## Pseudo Code - RGB

1. Generate random numbers
2. Interpret those values as RGB values.
3. Save the images as a PNG
4. Repeat


In [5]:
import pathlib
import progressbar as pb # type: ignore
import random
import shutil
import sys
from PIL import Image

# the size and count of the images
w = 480
h = 320
n = 50000
seed = 0

# setup folder
def ensure_folder(path: pathlib.Path) -> None:
    if path.exists():
        shutil.rmtree(path)
    path.mkdir(parents = True)

# fill the image with static
def generate_static_yiq(img: Image.Image, depth: str) -> None:
    y_rng = [0, 255]
    i_rng = [-151.9545, 151.9545]
    q_rng = [-133.2885, 133.2885]
    for w in range(img.width):
        for h in range(img.height):
            y = random.randint(y_rng[0], y_rng[1])
            if depth == 'bw':
                i = q = 0                
            elif depth == 'color':
                i = random.uniform(i_rng[0], i_rng[1])
                q = random.uniform(q_rng[0], q_rng[1])                
            rgb = yiq_to_rgb((y, i, q))
            img.putpixel((w, h), rgb)
def generate_static_rgb(img: Image.Image, depth: str) -> None:
    r_rng = [0, 255]
    g_rng = [0, 255]
    b_rng = [0, 255]
    for w in range(img.width):
        for h in range(img.height):
            r = random.randint(r_rng[0], r_rng[1])
            if depth == 'bw':
                g = b = r                
            elif depth == 'color':
                g = random.randint(g_rng[0], g_rng[1])
                b = random.randint(b_rng[0], b_rng[1])                
            img.putpixel((w, h), (r, g, b))

color_space = {'yiq': generate_static_yiq, 'rgb': generate_static_rgb}
color_depth = ['bw', 'color']

# run the loop
for space in color_space:
    for depth in color_depth:
        # consistency
        random.seed(seed)
        # calculated settings
        path = pathlib.Path(f'/data/notebook_files/{space}/{depth}')
        ensure_folder(path)
        print(f'--- {n} {w}x{h} {depth} images from {space} (seed {seed}) ---')
        widgets = [pb.Counter(), ' ', pb.Timer(), ' ', pb.BouncingBar(marker = '.', left = '[', right = ']')]
        with pb.ProgressBar(widgets = widgets, line_breaks = False, fd = sys.stdout) as bar:
            bar.update(0) # type: ignore
            for cnt in range(n):
                with Image.new(mode = "RGB", size = (w, h)) as img:
                    color_space[space](img, depth)
                    img.save(path.joinpath(f'{cnt:05d}.png'))
                    bar.update(cnt) # type: ignore

del w, h, n, seed, color_space, color_depth, space, depth, path, widgets, bar, img, cnt